<a href="https://colab.research.google.com/github/JEEVAGANN/AI/blob/main/pytorch_vs_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import tensorflow as tf
from torch.utils.data import DataLoader
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [10]:
# Load and preprocess the MNIST dataset using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

In [11]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [12]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [13]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [14]:
# Define the neural network using PyTorch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [15]:
model_pt = Net()
criterion = nn.CrossEntropyLoss()
optimizer_pt = optim.Adam(model_pt.parameters(), lr=0.001)

In [16]:
import time
start_time_tf = time.time()
model_tf = models.Sequential()
model_tf.add(layers.Flatten(input_shape=(28, 28, 1)))
model_tf.add(layers.Dense(128, activation='relu'))
model_tf.add(layers.Dense(10, activation='softmax'))

In [17]:
model_tf.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [18]:
model_tf.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time_tf = time.time()

Epoch 1/5
750/750 [==============================] - 5s 5ms/step - loss: 4.2688 - accuracy: 0.8610 - val_loss: 0.8065 - val_accuracy: 0.8894
Epoch 2/5
750/750 [==============================] - 5s 6ms/step - loss: 0.5284 - accuracy: 0.9001 - val_loss: 0.4576 - val_accuracy: 0.9135
Epoch 3/5
750/750 [==============================] - 3s 5ms/step - loss: 0.3177 - accuracy: 0.9262 - val_loss: 0.4355 - val_accuracy: 0.9199
Epoch 4/5
750/750 [==============================] - 3s 5ms/step - loss: 0.2627 - accuracy: 0.9361 - val_loss: 0.4282 - val_accuracy: 0.9302
Epoch 5/5
750/750 [==============================] - 4s 6ms/step - loss: 0.2308 - accuracy: 0.9412 - val_loss: 0.3511 - val_accuracy: 0.9410


In [19]:
print(f"TensorFlow Training Time: {end_time_tf - start_time_tf} seconds")

TensorFlow Training Time: 72.85971593856812 seconds


In [20]:
start_time_pt = time.time()
for epoch in range(5):
    for images, labels in train_loader:
        optimizer_pt.zero_grad()
        outputs = model_pt(images.view(-1, 28 * 28))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_pt.step()
end_time_pt = time.time()
print(f"PyTorch Training Time: {end_time_pt - start_time_pt} seconds")

PyTorch Training Time: 94.06459856033325 seconds


In [21]:
# Evaluation using TensorFlow
test_loss_tf, test_acc_tf = model_tf.evaluate(test_images, test_labels)
print(f"TensorFlow Test Accuracy: {test_acc_tf}")

313/313 [==============================] - 1s 2ms/step - loss: 0.3614 - accuracy: 0.9392
TensorFlow Test Accuracy: 0.9391999840736389


In [22]:
# Evaluation using PyTorch
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model_pt(images.view(-1, 28 * 28))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_pt = correct / total
print(f"PyTorch Test Accuracy: {accuracy_pt}")

PyTorch Test Accuracy: 0.9544
